# Cdiscount ConvNet with Keras

In [1]:
import os
import glob
from scipy import misc
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm_notebook as tqdmn

Using TensorFlow backend.


## Import the data into numpy array

In [2]:
img_width = 180
img_height = 180
n_channel = 3
train_folder = 'input/train'
validation_folder = 'input/validation'
test_folder = 'input/test'

class_list = os.listdir(train_folder)
n_classes = len(class_list)
n_train_examples = 12371293 #adapt
n_test_examples = 3095080



In [3]:
N_TRAINING_PER_CLASS

NameError: name 'N_TRAINING_PER_CLASS' is not defined

In [13]:
K.set_image_dim_ordering('th')
if K.image_data_format() == 'channels_first':
    input_shape = (n_channel, img_width, img_height)
else:
    input_shape = (img_width, img_height, n_channel)
    
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [7]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'input/train',  
        target_size=(img_height, img_width), 
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'input/validation',
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

Found 8659498 images belonging to 5270 classes.
Found 3711795 images belonging to 5270 classes.


In [14]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=test_generator,
        validation_steps=800 // batch_size)
model.save_weights('convnet_weights_1.h5')

Epoch 1/50


ValueError: Error when checking target: expected activation_25 to have shape (None, 5270) but got array with shape (16, 1)